Code to Delete all rows from the table

In [ ]:
import datetime
import yfinance as yf
import finnhub
import json
import time
import pandas as pd
from supabase import create_client

SUPABASE_URL = "url"
SUPABASE_KEY = "key"
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

FINNHUB_KEY = "api_key"
finnhub_client = finnhub.Client(api_key=FINNHUB_KEY)

def delete_existing_data():
    print("🗑️ Deleting all existing data from stock_data...")
    try:
        response = supabase.table("stock_data").delete().neq("symbol", "").execute()
        print("✅ All data deleted successfully.")
    except Exception as e:
        print(f"❌ Error deleting existing data: {str(e)}")

def fetch_historical_data(symbol):
    print(f"📊 Fetching historical data for {symbol}...")
    try:
        df = yf.download(symbol, period="1y", interval="1d")
        
        df.reset_index(inplace=True)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        
        df.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume"}, inplace=True)
        
        df["symbol"] = symbol
        df["date"] = df["date"].astype(str)
        df["open"] = df["open"].astype(float)
        df["high"] = df["high"].astype(float)
        df["low"] = df["low"].astype(float)
        df["close"] = df["close"].astype(float)
        df["volume"] = df["volume"].fillna(0).astype(int)

        return df.to_dict(orient='records')
    except Exception as e:
        print(f"⚠️ Error fetching data for {symbol}: {str(e)}")
        return None

def fetch_real_time_data(symbol):
    print(f"📈 Fetching real-time data for {symbol}...")
    try:
        if "-USD" in symbol:
            ticker = yf.Ticker(symbol)
            df = ticker.history(period="1d")
            if df.empty:
                raise ValueError("Yahoo Finance returned empty data for crypto")
            real_time_data = {
                "symbol": symbol,
                "date": datetime.datetime.now(datetime.UTC).isoformat(),
                "open": float(df["Open"].iloc[-1]),
                "high": float(df["High"].iloc[-1]),
                "low": float(df["Low"].iloc[-1]),
                "close": float(df["Close"].iloc[-1]),
                "volume": int(df["Volume"].iloc[-1]) if not df["Volume"].isna().iloc[-1] else 0
            }
        else:
            quote = finnhub_client.quote(symbol)
            volume_data = int(yf.Ticker(symbol).history(period='1d')["Volume"].iloc[-1])  
            real_time_data = {
                "symbol": symbol,
                "date": datetime.datetime.now(datetime.UTC).isoformat(),
                "open": quote["o"],
                "high": quote["h"],
                "low": quote["l"],
                "close": quote["c"],
                "volume": volume_data
            }
        return real_time_data
    except Exception as e:
        print(f"❌ Error fetching real-time data for {symbol}: {str(e)}")
        return None

def insert_to_supabase(data):
    try:
        if isinstance(data, list):
            for batch in [data[i:i + 100] for i in range(0, len(data), 100)]:
                response = supabase.table("stock_data").insert(batch).execute()
        else:
            response = supabase.table("stock_data").upsert([data], on_conflict=["symbol", "date"]).execute()
        print(f"✅ Successfully inserted {len(data) if isinstance(data, list) else 1} rows into stock_data")
    except Exception as e:
        print(f"❌ Error inserting data into stock_data: {str(e)}")

def main():
    symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "TSLA", "SPY", "QQQ", "BTC-USD", "ETH-USD"]
    
    for symbol in symbols:
        hist_data = fetch_historical_data(symbol)
        if hist_data:
            insert_to_supabase(hist_data)
        
        real_time_data = fetch_real_time_data(symbol)
        if real_time_data:
            insert_to_supabase(real_time_data)
    
    print("⏳ Waiting 5 minutes for next update...")
    time.sleep(300)

if __name__ == "__main__":
    while True:
        main()


❌ Error deleting data: name 'supabase' is not defined


💀❌ Code to delete all rows from your database

    Run function below


In [2]:
#delete_existing_data()